# MRR avec système de recommandation BPR sur les repas 

Cette partie de code est un complément du notebook principal sur le déjeuenr et le diner :
=> correpond au système de recommandation BPR (factorisation matricielle) pour les repas du déjeuner et le diner, avec optimisation du modèle sur les hyperparamètres 
=>  a été sur google colab surt gogle colab pour pouvoir utiliser la bpr loss qui ne fonctionnait pas autrement
=> objectif : avoir la performance en termes de ranking (MRR) sur un système de référénce BPR 

(Attention: les problèmes d'indentations en rouge sont dus au passage sur google colab, il faut les corriger ici)

In [ ]:
!pip install lightfm
import numpy as np
from lightfm import LightFM
import pickle
from lightfm import LightFM
from lightfm.data import Dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=879163 sha256=aa42ef901abbc3a2c1fcfaff5471500b80e3aab3a68ccd9de91bf79def667e9d
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [ ]:
#Catégories d'aliments crées par connaissance expert (aliments substituables) car le clustering avait échoué 
# on a retiré la catégorie c1 puisque ce sont des aliments/groupes d'alimenrs trop peu fréquemment consommés
import copy
c1=['2', '5', '6', '7', '9', '16', '19', '22', '25', '27', '28', '29', '35', '37', '40', '42', '44']
c2=['36', '3', '4', '39', '24']
c3=['10']
c4=['43']
c5=['17', '18', '21', '12']
c6=['11']
c7=['31', '32']
c8=['38']
c9=['23']
c10=['1']
c11=['33']
c12=['34']
c13=['30']
c14=['41', '26', '8']
c15=['13']
c16=['14']
c17=['15']
c18=['20']

c=[c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14, c15, c16, c17, c18]

c_new=copy.deepcopy(c)
c_new.remove(c1)

In [ ]:
with open("/d_indiv_alimfreq_6j.pkl", "rb") as fichier:
    d_indiv_alimfreq_6j = pickle.load(fichier)
with open("/Alim_non_rare_repas.pkl", "rb") as fichier:
    Alim_non_rare= pickle.load(fichier)
with open("/dic_alimnonrare_cat_repas.pkl", "rb") as fichier:
    dic_alimnonrare_cat= pickle.load(fichier)
with open("/dic_alimnonrare_cat_noms_repas.pkl", "rb") as fichier:
    dic_alimnonrarecat_names= pickle.load(fichier)
with open("/d_1j_rep.pkl", "rb") as fichier:
    d_1j= pickle.load(fichier)
with open("/d_alparcluster_1j.pkl", "rb") as fichier:
   d_alparcluster_1j= pickle.load(fichier)


# Initialize variables
users = list(d_indiv_alimfreq_6j.keys())
items = Alim_non_rare

# Create an empty list to store the interaction data
interaction_data = []

# Iterate over each user and calculate the frequency of food consumption
for user in users:
  for item in items:
        if str(item) in d_indiv_alimfreq_6j[user]:
           interaction_data.append((user, item))  # User has consumed the item

print(interaction_data)

# Create a Dataset object
dataset = Dataset()

# Fit the dataset with user and item IDs
dataset.fit(users, items)

# Build the interaction matrix
train_data, _ = dataset.build_interactions(interaction_data)
# print(train_data)



# Définir les différentes valeurs d'hyperparamètres à tester
epochs = [10, 1000, 5000, 9000]
no_components = [10, 50, 100, 200]
learning_rates = [5, 1, 0.01, 0.001]


best_mrr = 0.0
best_params = {}



for epoch in epochs:
  for num_components in no_components:
    for learning_rate in learning_rates:
      dic_bpr={k: {tuple(i): [] for i in c_new} for k in list(d_indiv_alimfreq_6j.keys())}
            # Créer un modèle LightFM avec les hyperparamètres actuels
      model = LightFM(loss='bpr', learning_rate=learning_rate, no_components=num_components)
         # Entraîner le modèle sur les données d'entraînement
      model.fit(train_data, epochs=epoch, num_threads=1)

        # Évaluer MRR sur les données de validation
      total_mrr=0
      for user_idx, user_id in enumerate(users):
        train_prediction=model.predict(user_idx, list(range(len(items))))
           # print(train_prediction)
        items_tries = sorted(range(len(train_prediction)), key=lambda i: train_prediction[i], reverse=True)
           # print(items_tries)
        alim_tries = [Alim_non_rare[element] for element in items_tries]
        # print(alim_tries)
        for e in alim_tries:
          for i in c_new:
            if e in dic_alimnonrare_cat[tuple(i)]:
              dic_bpr[user_id][tuple(i)].append(e)
              # print(dic_bpr[user_id])
        real_conso=d_alparcluster_1j[user_id]
              # print(real_conso)

        mrr = 0
        nbre_alim=0
        for k in c_new:
          for consumption in real_conso[tuple(k)]:
            mrr_e=0
            nbre_alim+=1
            if int(consumption.replace("'","")) in dic_bpr[user_id][tuple(k)]:
              rank = dic_bpr[user_id][tuple(k)].index(int(consumption.replace("'","")))
              inverse_rank = 1/(rank+1)
              mrr += inverse_rank
        mrr /= nbre_alim
        # print("mrr id", user_id, mrr)
        total_mrr+=mrr

      performance=total_mrr/len(d_indiv_alimfreq_6j.keys())
      print("MRR moyenne sur tous les utilisateurs:", total_mrr/len(d_indiv_alimfreq_6j.keys()))

      if performance > best_mrr:
        best_mrr = performance
        best_params = {
          'no_components': num_components,
          'learning_rate': learning_rate,
          'epochs': epoch
          }



print("Meilleurs hyperparamètres :")
print(best_params)
print("Meilleure performance (MRR) :", best_mrr)

[(110006, 28301), (110006, 9811), (110006, 7001), (110006, 18008), (110006, 19611), (110006, 13030), (110006, 16632), (110006, 81007), (110006, 20047), (110006, 11110), (110006, 22501), (110006, 39221), (110006, 25163), (110006, 19640), (110006, 13037), (110006, 8400), (110006, 25103), (110006, 9104), (110006, 36005), (110006, 20071), (110006, 13014), (110006, 28925), (110006, 36306), (110006, 20101), (110006, 4003), (110006, 17440), (110006, 28504), (110006, 11081), (110006, 28900), (110006, 19624), (110020, 9811), (110020, 13039), (110020, 19700), (110020, 76022), (110020, 11114), (110021, 7001), (110021, 18008), (110021, 25103), (110021, 9104), (110021, 25902), (110021, 12114), (110021, 81009), (110021, 81029), (110025, 9811), (110025, 36005), (110025, 13034), (110025, 81024), (110025, 81061), (110025, 11107), (110025, 81022), (110025, 76061), (110025, 20051), (110034, 7001), (110034, 13037), (110034, 4003), (110034, 5205), (110034, 25902), (110034, 13039), (110034, 90068), (110034,